<a href="https://colab.research.google.com/github/nhareesha/MLAI/blob/main/LLM/finetuned/Attempt_7_data_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

csv_path = '/content/drive/MyDrive/diverse_test_data_01.csv'  # Replace with your desired path


Mounted at /content/drive


In [3]:
pip install transformers

In [4]:
import csv
from torch.utils.data import Dataset
# from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
# from transformers import ConvBertForSequenceClassification
from transformers import Trainer, TrainingArguments
# from transformers import CodeGenPreTrainedModel, CodeGenTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import random


In [12]:
import random

In [5]:

class SQLDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=512, prompt_prefix="Generate an INSERT query for the following table schema:"):
        self.file_path = file_path
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.prompt_prefix = prompt_prefix
        self.data = self.load_data()

    def load_data(self):
        data = []
        with open(self.file_path, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                ddl = row['ddl']
                insert = row['insert']
                data.append((ddl, insert))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        ddl, insert = self.data[idx]
        prefixed_ddl = self.prompt_prefix + "\n" + ddl
        inputs = self.tokenizer(ddl, max_length=self.max_length, truncation=True, padding='max_length', return_tensors="pt")
        labels = self.tokenizer(insert, max_length=self.max_length, truncation=True, padding='max_length', return_tensors="pt")

        # Flatten the tensor dimensions
        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        labels = labels.input_ids.squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
            }


# Train using Huggingface trainer API

In [6]:

# Load the tokenizer and model
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#model = GPT2LMHeadModel.from_pretrained('gpt2')
#tokenizer.pad_token = tokenizer.eos_token

# tokenizer = CodeBertTokenizer.from_pretrained("microsoft/codebert-base")
# model = CodeBertForSequenceClassification.from_pretrained("microsoft/codebert-base")
# tokenizer.pad_token = tokenizer.eos_token

# tokenizer = CodeGenTokenizer.from_pretrained("microsoft/codegen-35m")
# model = CodeGenPretrainedModel.from_pretrained("microsoft/codegen-35m")

# tokenizer = AutoTokenizer.from_pretrained("microsoft/codegen-35m")
# model = AutoModelForCausalLM.from_pretrained("microsoft/codegen-35m")
# tokenizer.pad_token = tokenizer.eos_token

# tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
# model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codegen-350M-mono")

# Define device based on GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("t5-small")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Move model and inputs to the device (when you define them)
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Split datasets

In [16]:
# Load your CSV data assuming it's named 'diverse_test_data.csv'
data = []
with open(csv_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['ddl'], row['insert']))

# Split data into training and evaluation sets (e.g., 80%/20%) - HEAVY
# train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)

# Randomly sample 50k records
random.shuffle(data)  # Shuffle the data for randomness
train_data = data[:20000]  # Take the first 50k records for training
eval_data = data[20000:25000]  # Use the remaining records for evaluation


# Write training and evaluation data to separate CSV files
train_csv_path = 'train_data.csv'
eval_csv_path = 'eval_data.csv'

with open(train_csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ddl', 'insert'])
    writer.writerows(train_data)

with open(eval_csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ddl', 'insert'])
    writer.writerows(eval_data)


# Create separate datasets for training and evaluation
train_dataset = SQLDataset(train_csv_path, tokenizer)
eval_dataset = SQLDataset(eval_csv_path, tokenizer)

In [17]:
# Initialize the SQLDataset
# dataset = SQLDataset(file_path=csv_path, tokenizer=tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results_base', # change to colab
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Adjust batch size based on hardware
    gradient_accumulation_steps=2,  # If needed to handle larger batches
    learning_rate=5e-5,  # Adjust learning rate as needed
    lr_scheduler_type='cosine',  # Consider using a learning rate scheduler
    save_steps=4000,
    save_total_limit=2,
    logging_dir='./logs_t5_base',
    logging_steps=500,
    evaluation_strategy='steps',
    eval_steps=2000,  # Reduce evaluation frequency
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset  # Add evaluation dataset for monitoring performance

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# Start training
trainer.train()

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
2000,1.132400,1.108782


TrainOutput(global_step=3750, training_loss=1.159366357421875, metrics={'train_runtime': 4963.438, 'train_samples_per_second': 12.088, 'train_steps_per_second': 0.756, 'total_flos': 8120508088320000.0, 'train_loss': 1.159366357421875, 'epoch': 3.0})

In [19]:
# Save the trained model
import os

# Create the directory if it doesn't exist
save_dir = "/content/drive/MyDrive/saved_model/ddl-to-insert_t5_small"
os.makedirs(save_dir, exist_ok=True)

# Save the trained model
trainer.save_model(save_dir)


In [24]:
import torch

def generate_insert_query(ddl, model, tokenizer, max_length=1000):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inputs = tokenizer(ddl, return_tensors="pt", return_attention_mask=True)
    inputs = inputs.to(device)
    tokenizer.pad_token_id = tokenizer.eos_token_id
    # outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=max_length, num_return_sequences=1)
    outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=max_length, num_return_sequences=1, do_sample=True, top_k=50, top_p=0.9,temperature=0.7)


    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [26]:
ddl_input = "Generate an INSERT query for the following table schema:CREATE TABLE employees (id INT, first_name VARCHAR(100), last_name VARCHAR(100), department VARCHAR(100), salary DECIMAL(10, 2));"
insert_query = generate_insert_query(ddl_input, model, tokenizer)

print("Generated INSERT query:", insert_query)

Generated INSERT query: INSERT INTO table schema.


In [29]:
ddl_input = "CREATE TABLE employees(id INT, first_name VARCHAR(100), last_name VARCHAR(100), department VARCHAR(100), salary DECIMAL(10, 2));"
insert_query = generate_insert_query(ddl_input, model, tokenizer)

print("Generated INSERT query:", insert_query)

Generated INSERT query: INSERT INTO employees (id, never, his, the, department, 58.9019), (i.e, the owl, staff, 7702.68), (blk, 7797.678688, (snc;v, show, do, us, -75.95), (ie, be, -29.873917, show, department, 79.8793), (ie, 84.66, think, department, 8208.8032), -8973, ty, hqaop, -3625.902);


In [30]:
ddl_input = "CREATE TABLE orders(id INT, name VARCHAR(100), order_date DATE, fullfill_date DATE);"
insert_query = generate_insert_query(ddl_input, model, tokenizer)

print("Generated INSERT query:", insert_query)

Generated INSERT query: INSERT INTO MANUFACTORIES (2248, afmsk, 2001-06-23, 2010-07-09);


In [35]:
ddl_input = "CREATE TABLE table_200(column_1 INT, column_2 VARCHAR(100), column_3 DATE, column_4 DECIMAL(5,2));"
insert_query = generate_insert_query(ddl_input, model, tokenizer)

print("Generated INSERT query:", insert_query)

Generated INSERT query: INSERT INTO table_200 VALUES (2395.375, full, 2015-02-04, -3338.70);
